In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import re, sys, random, glob, json, unidecode, unicodedata
import ipdb, time, sys, os, pickle
from IPython.core.display import display, HTML
from IPython.display import clear_output
from urllib.request import urlopen
from collections import Counter
from bs4 import BeautifulSoup  

display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib Configuration
pylab.rcParams.update({'legend.fontsize': 'medium',
                       'font.size'      : 12,
                       'figure.figsize' : (10, 10),
                       'axes.labelsize' : 'medium',
                       'axes.titlesize' : 'medium',
                       'axes.grid'      : 'on',
                       'xtick.labelsize': 'medium',
                       'ytick.labelsize': 'medium'})

def print_runtime(start: float, printer: bool = True) -> None:
    end = time.time()
    if printer:
        print(f'Runtime: {int((end - start) // 60)} min {int((end - start) % 60):2d} sec')
    else:
        return f'({int((end - start) // 60)} min {int((end - start) % 60):2d} sec)'

/var/folders/jf/qykqz6ks0vs56cb_zyf_7x_00000gn/T/ipykernel_1248/4190775106.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


1222

---
# Extract names from emails

In [335]:
fname = 'First Name Change - Interexo 1_29_24.xlsx - Data - No Change.csv'
df = pd.read_csv(fname)
df.insert(1, 'likelihood', [0]*len(df))
df.rename({'Given Name': 'Preferred Name'}, axis=1, inplace=True)
df = df.astype({'Preferred Name':str, 'likelihood':int})
df.head()

,Preferred Name,likelihood,First Name,Last Name,Email,Company Name,Bio,Address,Country,State,City,Zip Code,Phone #,Phone #.1,Website,Profile Urls
0,nan,0,Christopher,Cheung,chris.cheung@fbxbrokers.com,Florida Business Exchange,Chris Cheung is a \t\t\t\t\t\tTampa business b...,550 Reo St.\t\t\t\t\t\t\t\t\t\t\t\t\tSuite 300,United States,FL,Tampa,33602,813-785-9089,NaN,https://fbxbrokers.com,https://brokers.interexo.com/united-states_flo...
1,nan,0,William,Gaines,bill@franpro.com,Franprocom,William Gaines is a business broker of the we...,Bowie,United States,MD,20715,NaN,443-583-3070,NaN,https://franpro.com,https://brokers.interexo.com/united-states_mar...
2,nan,0,Anthony,Shard,tony@shardmb.com,Shard Marketing,Anthony Shard is a \t\t\t\t\t\tCortez business...,12325 Baypointe Ter,United States,FL,Cortez,34215-2566,810-577-5291,NaN,https://shardmb.com,https://brokers.interexo.com/united-states_flo...
3,nan,0,Antonio,Bardy,tonybardy@gmail.com,Florida Connexion Properties,Antonio Bardy is a \t\t\t\t\t\tWindermere busi...,1961 Westhill Run,United States,FL,Windermere,34786-6230,407-535-3639,NaN,NaN,https://brokers.interexo.com/united-states_flo...
4,nan,0,Elizabeth,Berger,liz@lizberger.com,Berger Realty Group,Elizabeth Berger is a Coral Springs business b...,1955 University Dr,United States,FL,Coral Springs,33071,954-344-8222,954-650-7848,https://lizberger.com/,https://brokers.interexo.com/united-states_flo...


In [450]:
from openai import OpenAI
client = OpenAI()

start = time.time()
model = "gpt-3.5-turbo-1106"

sys_message = f"""You are a helpful assistant. You job is, given the "Given Name", "Email" and "Bio" of a company employee, to estimate what his "Preferred Name" is. For instance, a person with the given name 'Anthony' might prefer being called 'Tony', and the evidence of it could be that his email was 'tony_pearson@gmail.com'. However, if you have no evidence for his "Preferred Name" being different than his "Given Name", just set the "Preferred Name" to be equal to his Given Name.
Provide your completion in a JSON format with only two keys like this: """+\
 '{"Preferred Name":val_1, "likelihood": val_2}.' +\
f"""The value of the "Preferred Name" (val_1) should only include the estimated "preferred name" that can be used to replace the "Given Name", and nothing else. The value of "likelihood" (val_2) should have a single integer number value between 0 and 100 for that quantifies the certainty of your estimate of the "Preferred Name",  where 0 means you're not certain about your prediction at all, and 100 means you're absolutely certain about it.  In your response, do not include any other text aside froim the JSON output, any extra explanation or any special formatting like using backticks (``).
    
I'm providing the "Given Name", "Email", and the "Bio" of a business professional. Based on this information, let me know what you think the said individual prefers to be called. """

def get_prompt(first_name, email, bio):
    prompt = f"""

    Given Name: {first_name}
    Email: {email}
    Bio: {bio}

 """
    return prompt

p = 1002
for idx, row in df[p:p+1].iterrows():
    print(f"{idx:3d}   Name: {row['First Name']} ", end=" ")
    
    prompt = get_prompt(row['First Name'], row['Email'], row['Bio'])
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": f"""You are a helpful assistant. {sys_message}"""},
            {"role": "user", "content": prompt}
        ],
        temperature=0.01,
        logprobs=True,
        top_logprobs=5
    )
    
    content = response.choices[0].message.dict()['content']
    content = content.replace(',}', '}').replace(",\n", ",")
    json_output = json.loads(content)
    
    df.loc[idx, 'Preferred Name'] = json_output['Preferred Name']
    df.loc[idx, 'likelihood'] = int(json_output['likelihood'])
    print(' --- ', json_output['Preferred Name'], '\n','----'*17)
    

1002   Name: Dennis   ---  Denny 
 --------------------------------------------------------------------


In [452]:
print(len(response.choices[0].logprobs.content))
token


13


TopLogprob(token='Don', bytes=[68, 111, 110], logprob=-13.91887)

In [454]:
for token in response.choices[0].logprobs.content[5].top_logprobs:
    print(f'{token.token:6s} {100 * np.exp(token.logprob):.2f}')
    

enny   94.70
ennis  5.29
enn    0.00
ennie  0.00
","    0.00
